In [26]:
import pandas as pd
import gzip
from tqdm.notebook import tqdm

In [2]:
text_emb = pd.read_csv('../output/text-embeddings/missing_nodes.tsv', sep='\t')

In [6]:
text_emb.node1.unique()[:20]

array(['Q1000959', 'Q100156149', 'Q100276118', 'Q100287557', 'Q100306102',
       'Q100325978', 'Q100345932', 'Q100347932', 'Q100347974',
       'Q100348014', 'Q100348021', 'Q100348032', 'Q100348054',
       'Q100348078', 'Q100385591', 'Q100385624', 'Q100385642',
       'Q100385667', 'Q100388742', 'Q100388938'], dtype=object)

In [10]:
text_emb.label.unique()

array(['P279', 'P31', 'P373', 'P452', 'P1382', 'P106'], dtype=object)

In [13]:
subset_nodes = set(['Q1000959', 'Q100156149', 'Q100276118', 'Q100287557', 'Q100306102',
       'Q1000959', 'Q105046695', 'Q105416259', 'Q105135334'])

In [17]:
text_emb = text_emb[text_emb.node1.apply(lambda p: p in subset_nodes)].reset_index().drop(columns=['index'])

In [33]:
text_emb.to_csv('../TEMP/kgtk_unittest/text_embedding.edges.tsv', sep='\t', index=None)

In [24]:
def get_labels(node_set):
    labels_dict = {}
    lines = []
    first_line = None
    with gzip.open("../source_dataset_files/wikidata-20210215-dwd-v2/labels.en.tsv.gz", 'r') as labelsFile:
        for ogline in tqdm(labelsFile, total=41845781):
            if first_line:
                first_line = False
                lines.append(ogline)
                continue
            line = ogline.decode('utf-8').strip().split('\t')
            line[3] = line[3][1:-5]
            qnode, label = line[1], line[3]
    #         print(qnode, label)
            if qnode in node_set:
                lines.append(ogline)
    return lines

In [45]:
lines = get_labels(set(text_emb.node1.to_list() + text_emb.node2.to_list()))

  0%|          | 0/41845781 [00:00<?, ?it/s]

In [46]:
lines = [line.decode('utf-8') for line in lines]

In [47]:
lines = ['id\tnode1\tlabel\tnode2\n'] + lines

In [48]:
with open('../TEMP/kgtk_unittest/text_embedding.labels.tsv', 'w') as f:
    f.writelines(lines)

In [63]:
pd.read_csv('~/kgtk/tests/data/text_embedding.edges.tsv', sep='\t').node1.nunique()

8

In [71]:
temp1 = pd.read_csv('~/kgtk/tests/data/out.tsv', sep='\t')
temp2 = pd.read_csv('~/kgtk/tests/data/text_embedding.output.tsv', sep='\t')

In [72]:
temp1

,node,property,value
0,Q105135334,text_embedding,"-0.6744056,-0.2023162,-0.27134913,-0.13181135,..."
1,Q105135334,embedding_sentence,NaN
2,Q105046695,text_embedding,"-0.6744056,-0.2023162,-0.27134913,-0.13181135,..."
3,Q105046695,embedding_sentence,NaN
4,Q100306102,text_embedding,"-0.6744056,-0.2023162,-0.27134913,-0.13181135,..."
5,Q100306102,embedding_sentence,NaN
6,Q100287557,text_embedding,"-0.6744056,-0.2023162,-0.27134913,-0.13181135,..."
7,Q100287557,embedding_sentence,NaN
8,Q100276118,text_embedding,"-0.6744056,-0.2023162,-0.27134913,-0.13181135,..."
9,Q100276118,embedding_sentence,NaN


In [70]:
temp2

,node,property,value
0,Q105416259,text_embedding,"0.062068786,-0.04527851,0.08445225,0.020385189..."
1,Q105416259,embedding_sentence,"It is Content creation, group of humans."
2,Q105135334,text_embedding,"-0.5932912,0.15151715,0.12169738,-0.57092476,0..."
3,Q105135334,embedding_sentence,"It is costume component, hatband."
4,Q105046695,text_embedding,"0.78254557,0.7347305,-0.24301007,-0.40795732,0..."
5,Q105046695,embedding_sentence,"It is field of work, health care industry, ser..."
6,Q100306102,text_embedding,"1.0986179,0.4818771,-0.46546745,0.28162912,1.0..."
7,Q100306102,embedding_sentence,"It is bridged compound, pair of enantiomers."
8,Q100287557,text_embedding,"0.91020757,0.6681585,-0.3698121,2.749119,1.556..."
9,Q100287557,embedding_sentence,"It is mushroom dish, veggie burger."


In [67]:
!~/miniconda3/envs/kgtkEnv2/bin/kgtk text-embedding -i ~/kgtk/tests/data/text_embedding.edges.tsv   \
    --model roberta-large-nli-mean-tokens   \
    --property-labels-file ~/kgtk/tests/data/text_embedding.labels.tsv  --debug   \
    --isa-properties P31 P279 P106 P39 P1382 P373 P452   \
    --save-embedding-sentence --out-file ../output/text-embeddings/P279-text-embedding-7-props-missing-qnodes.tsv

/nas/home/kshenoy/miniconda3/envs/kgtkEnv2/lib/python3.8/site-packages/rdflib_jsonld/__init__.py:9: DeprecationWarning: The rdflib-jsonld package has been integrated into rdflib as of rdflib==6.0.1.  Please remove rdflib-jsonld from your project's dependencies.
  warnings.warn(
Batches:   0%|                                            | 0/1 [00:00<?, ?it/s]/nas/home/kshenoy/miniconda3/envs/kgtkEnv2/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")
Batches: 100%|████████████████████████████████████| 1/1 [00:00<00:00, 18.93it/s]


In [59]:
!cp ../TEMP/kgtk_unittest/text_embedding.edges.tsv ~/kgtk/tests/data/text_embedding.edges.tsv

In [58]:
!grep "sentence" ~/kgtk/tests/data/text_embedding.output.tsv

Q105416259	embedding_sentence	It is Content creation, group of humans.
Q105135334	embedding_sentence	It is costume component, hatband.
Q105046695	embedding_sentence	It is field of work, health care industry, service, transportation industry.
Q100306102	embedding_sentence	It is bridged compound, pair of enantiomers.
Q100287557	embedding_sentence	It is mushroom dish, veggie burger.
Q100276118	embedding_sentence	It is group of stereoisomers, sugar acid.
Q100156149	embedding_sentence	It is food brand, meat analogue.
Q1000959	embedding_sentence	It is Reanda (apple), apple, apple cultivar.


In [4]:
first_line = None
with open('../output/text-embeddings/missing_nodes.tsv') as f:
    for line in f:
        if first_line is None:
            first_line = line
            continue
        line1 = line